In [1]:
import xarray as xr
import yaml
import zarr
import numpy as np
import chunk_tools  # local package to outsource some basic functions

In [2]:
def to_zarr_store(path, outds, timechunk, order):
    store = zarr.storage.DirectoryStore(
        path, normalize_keys=False, dimension_separator="/"
    )
    outds.to_zarr(
        store,
        encoding=chunk_tools.get_encodings(
            outds=outds, timechunk=timechunk, order=order
        ),
    )  # , compute=False
    store.close()

In [3]:
def process_2d(curr_conf, ds):
    ds = ds.rename(curr_conf["renames"])
    ds = ds.isel(curr_conf["isel"])
    return ds

In [4]:
def convert_files(zoom, subset):
    files = (
        f"/large/sftpgo/data/NICAM/hackathon/tksk_sample/*/{subset}/z{zoom:02d}/*.nc"
    )
    outfile = f"{output_dir}/NICAM_{subset}_z{zoom}.zarr"
    curr_conf = config[subset]
    timechunk = curr_conf["chunks"]["time"]

    ds = xr.open_mfdataset(files, chunks=curr_conf["chunks"])
    out_ds = process_2d(curr_conf, ds)

    ! rm -rf {outfile}
    to_zarr_store(path=outfile, outds=out_ds, timechunk=timechunk, order=zoom)
    return outfile

In [5]:
output_dir = "/large/work/florian"
config = yaml.safe_load(open("nicam_to_zarr.yaml"))
for subset in ("2d1h",):
    for zoom in range(10):
        outfile = convert_files(zoom=zoom, subset=subset)
        ds_new = xr.open_dataset(outfile)

In [6]:
ds_new

<xarray.Dataset> Size: 110GB
Dimensions:         (time: 264, cell: 3145728)
Coordinates:
    lev             float64 8B ...
  * time            (time) datetime64[ns] 2kB 2020-01-02T00:30:00 ... 2020-01...
Dimensions without coordinates: cell
Data variables: (12/34)
    clivi           (time, cell) float32 3GB ...
    clwvi           (time, cell) float32 3GB ...
    healpix         (time) int32 1kB ...
    hflsd           (time, cell) float32 3GB ...
    hfssd           (time, cell) float32 3GB ...
    huss            (time, cell) float32 3GB ...
    ...              ...
    sa_tem_atm      (time, cell) float32 3GB ...
    sa_tppn_energy  (time, cell) float32 3GB ...
    tas             (time, cell) float32 3GB ...
    ts              (time, cell) float32 3GB ...
    uas             (time, cell) float32 3GB ...
    vas             (time, cell) float32 3GB ...
Attributes:
    comment:  Be careful that definition of time coordinate depends on the da...
    history:  Generated by mod_netcdf.f90.
    title:    NICAM data output

In [7]:
!cat {outfile}/.zmetadata

{
    "metadata": {
        ".zattrs": {
            "comment": "Be careful that definition of time coordinate depends on the dataset and time mode (snapshot/average).",
            "history": "Generated by mod_netcdf.f90.",
            "title": "NICAM data output"
        },
        ".zgroup": {
            "zarr_format": 2
        },
        "clivi/.zarray": {
            "chunks": [
                24,
                65537
            ],
            "compressor": {
                "blocksize": 0,
                "clevel": 5,
                "cname": "zstd",
                "id": "blosc",
                "shuffle": 2
            },
            "dimension_separator": "/",
            "dtype": "<f4",
            "fill_value": "NaN",
            "filters": null,
            "order": "C",
            "shape": [
                264,
                3145728
            ],
            "zarr_format": 2
        },
        "clivi/.zattrs": {
            "_ARRAY_DIMENSIONS": [
                

In [8]:
ds_new

<xarray.Dataset> Size: 110GB
Dimensions:         (time: 264, cell: 3145728)
Coordinates:
    lev             float64 8B ...
  * time            (time) datetime64[ns] 2kB 2020-01-02T00:30:00 ... 2020-01...
Dimensions without coordinates: cell
Data variables: (12/34)
    clivi           (time, cell) float32 3GB ...
    clwvi           (time, cell) float32 3GB ...
    healpix         (time) int32 1kB ...
    hflsd           (time, cell) float32 3GB ...
    hfssd           (time, cell) float32 3GB ...
    huss            (time, cell) float32 3GB ...
    ...              ...
    sa_tem_atm      (time, cell) float32 3GB ...
    sa_tppn_energy  (time, cell) float32 3GB ...
    tas             (time, cell) float32 3GB ...
    ts              (time, cell) float32 3GB ...
    uas             (time, cell) float32 3GB ...
    vas             (time, cell) float32 3GB ...
Attributes:
    comment:  Be careful that definition of time coordinate depends on the da...
    history:  Generated by mod_netcdf.f90.
    title:    NICAM data output